# Recherche du meillieur algo ML - Entranement test - pipeline¶

In [18]:
#Importation des bibliothèques nécéssaires
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report
import joblib
from time import time


In [19]:
# fonction pour le chargement du dataSet à partir du fichier CSV
def train_test_split_fonction(method, filename):
    df= pd.read_csv(filename, sep=';')
    if method == "stem":
        X, y = df.StemmedComments, df.Sentiment
    if method == "lem":
        X, y = df.LemmatizedComments, df.Sentiment
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify = df.Sentiment)
    return X_train, X_test, y_train, y_test


In [20]:
def pipeline_fonction_avec_gridsearch(classifier, vectorizer):
    #  Paramètres GridSerach pour les vectorizers (Count and TFIDF)
    parameters_vect = {
        'vectorizer__max_df': [0.5, 0.75, 1.0],
        'vectorizer__ngram_range': [(1, 1), (1, 2), (1,3)],
        'vectorizer__max_features': [2000, 4000]
    }
    
    
    # paramètres GridSearch pour les Classifieurs 
    if classifier == "log":
        parameters_clf = {"classifier__C":[0.01,0.1, 1, 10, 100],
                          "classifier__penalty":['l1', 'l2']}  
    elif classifier == "rf":
        parameters_clf = {'classifier__n_estimators' : [10, 50, 100,200,500],
                          'classifier__max_features': ['auto', 'log2', 'sqrt'], 
                          'classifier__criterion' :['gini', 'entropy']} 
    elif classifier == "svm" : 
        parameters_clf = {'classifier__kernel': ['linear', 'rbf'],
                          'classifier__C':[0.01,0.1, 1, 10, 100], 
                          'classifier__gamma' : [0.001, 0.0001]}
    else : 
        return "Le classifieur spécifié n'est pas traité !"
    
    
    # définition du vectorizer 
    if vectorizer == 'tfidf' : 
        vect =  TfidfVectorizer()
    elif vectorizer =='count' : 
        vect = CountVectorizer()
    else :
        return "Le vectorizer spécifié n'est pas traité !"
    
    
    # définition du classifier 
    if classifier == "log": 
        clf = LogisticRegression()
    elif classifier == "rf":
        clf = RandomForestClassifier()
    elif classifier == "svm" : 
        clf = SVC() 
    else : 
        return "Le classifieur spécifié n'est pas traité !"
    
    
    # Création du Pipeline
    # L'attribut "memory" permet d'activer la mise en cache des transformateurs du pipeline (CountVect/TFIDF) dans le dossier spécifié 
    # le tranformateur n'est pas calculé/entrainé à chaque fois, si les paramètres et les données d'entrée sont identiques (optimisation du temps de calcul) 
    cache_dir='./cachedir'
    
    sentiment_pipeline = Pipeline([
        ('vectorizer', vect),
        ('classifier', clf)
    ], memory=cache_dir)
    
    
    # Constuire l'objet GridSearchCV 
    # Définition de la liste compète des paramèters (hyperparamètres) de la GridSearchCV
    gridParameters = dict()
    gridParameters.update(parameters_vect)
    gridParameters.update(parameters_clf)
    
    
    # Création de l'objet GridSearch Associé 
    # P.S : n_jobs (nbre de processus à exécuter en parallèle ; n_jobs = -1 --> utiliser tous les processus du PC si celui-ci le permet)
    # P.S: Score d'évaluation "roc_auc" pour prendre en compte le taux de faux postifs, en plus du taux des vrais positifs
    #      --> Vue que le DataSet est déséquilibrée, cette métrique est mieux conseillée que l'accuracy pour la compraison des perf des algos. 
    # verbose = 1: pour afficher quelques message --> permet le suivi de l'avancement de l'algo
    gs = GridSearchCV(sentiment_pipeline, param_grid=gridParameters,scoring='roc_auc', verbose=1, cv=5, n_jobs=-1)

    # return l'objet GridSearch
    return gs


In [21]:
# Fichier CSV contenant la DataSet à traiter 
dataFile = 'all_reviews_Fr_Prepared_binaryClass.csv'  # Commentaires Français
#dataFile = 'all_reviews_En_Prepared_binaryClass.csv'  # Commentaires Anglais

# Load & Split the Data 
# Données avec Stemmatisation 
X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split_fonction("stem", dataFile)

X_train_lem, X_test_lem, y_train_lem, y_test_lem = train_test_split_fonction("lem", dataFile)

In [22]:
# On va comparer la performance de 4 algorithmes ML différents connus pour l'analyse du Texte (logisticRegression; RandomForest; SVM; NaiveBayes)
# Pour chaque algorithme , on fera appel à GridSearch pour optimiser ses hyperparamétres
# Et cela dans les deux cas de traitement de texte : Stemming & Lemmatization et les deux vectorizers utilisés  

# Création des différents pipelines nécéssaires 
# Avec Stemming
gs_stem_count_rf = pipeline_fonction_avec_gridsearch('rf', 'count')
gs_stem_tfidf_rf = pipeline_fonction_avec_gridsearch('rf', 'tfidf')
gs_stem_count_log = pipeline_fonction_avec_gridsearch('log', 'count')
gs_stem_tfidf_log = pipeline_fonction_avec_gridsearch('log', 'tfidf')
gs_stem_count_svm = pipeline_fonction_avec_gridsearch('svm', 'count')
gs_stem_tfidf_svm = pipeline_fonction_avec_gridsearch('svm', 'tfidf')
# Avec lemmatization 
gs_lem_count_rf = pipeline_fonction_avec_gridsearch('rf', 'count')
gs_lem_tfidf_rf = pipeline_fonction_avec_gridsearch('rf', 'tfidf')
gs_lem_count_log = pipeline_fonction_avec_gridsearch('log', 'count')
gs_lem_tfidf_log = pipeline_fonction_avec_gridsearch('log', 'tfidf')
gs_lem_count_svm = pipeline_fonction_avec_gridsearch('svm', 'count')
gs_lem_tfidf_svm = pipeline_fonction_avec_gridsearch('svm', 'tfidf')

# Enregsitrement des pipelines dans une liste pour automatiser les itérations
grids = [ gs_stem_count_rf,
         gs_stem_tfidf_rf,
         gs_stem_count_log, 
         gs_stem_tfidf_log,
         gs_stem_count_svm,
         gs_stem_tfidf_svm,
         gs_stem_count_nb,
         gs_stem_tfidf_nb,  
         gs_lem_count_rf,
         gs_lem_tfidf_rf,
         gs_lem_count_log,
         gs_lem_tfidf_log,
         gs_lem_count_svm,
         gs_lem_tfidf_svm
        ]

# Dictionnaire des pipelines et des types de classificateurs pour faciliter le référencement 
grid_dict = {0: 'Stemming - Random Forest/Countvectorizer',
             1: 'Stemming - Random Forest/TFIDF Vectorizer',
             2: 'Stemming - Logistic Regression/Countvectorizer',
             3: 'Stemming - Logistic Regression/TFIDF Vectorizer', 
             4: 'Stemming - Support Vector Machine/Countvectorizer',
             5: 'Stemming - Support Vector Machine/TFIDF Vectorizer',
             6: 'Lemmatization - Random Forest/Countvectorizer',
             7: 'Lemmatization - Random Forest/TFIDF Vectorizer',
             8: 'Lemmatization - Logistic Regression/Countvectorizer',
             9: 'Lemmatization - Logistic Regression/TFIDF Vectorizer', 
             10: 'Lemmatization - Support Vector Machine/Countvectorizer',
             11: 'Lemmatization - Support Vector Machine/TFIDF Vectorizer'
            }

In [ ]:
# Fit the grid search objects
print('********* Optimisation des Modèles ML et Réglages des Hyperparamètres ....**********')
best_roc_auc = 0.0
best_clf = 0
best_gs = ''

# Boucle pour l'évaluation de tous les pipelines (enregistrement du meilleure Classifieur pour chaque pipeline )
for idx, gs in enumerate(grids) : 
    
    print("\n Nouvelle instane GridSearch...")
    print('Pipeline : %s' % grid_dict[idx])
    print('------------------------------------------------')
    # Entrainement du Grid Search sur le data_train
    t0 = time()
    if (grid_dict[idx][0:3]=='Lem'): # fit on the LemmatizedComments
        gs.fit(X_train_lem, y_train_lem)
    else : 
        gs.fit(X_train_stem, y_train_stem)
    print("Entrainement SearchGrid faut %0.4f" % (time() - t0))
    # Meilleur pipeline 
    best_pipe = gs.best_estimator_
    # Meilleur paramètres et score 
    print('Meilleur Score CV d\'entrainement (ROC AUC): %.4f' % gs.best_score_)   
    print('Meilleur ensemble de paramètres: %s' % gs.best_params_)
    # Prédire les donnes de Test avec le meilleur classifieur
    if (grid_dict[idx][0:3]=='Lem'): # fit on the LemmatizedComments
        y_pred = best_pipe.predict(X_test_lem)
        roc_score = roc_auc_score(y_test_lem, y_pred)
        print('Score AUC ROC sur les données de Test avec le meilleur pipeline : %0.4f ' % roc_score )
        print('\n')
         # Afficher d'autres métriques significatives
        print('Score "Accuracy" sur les données de Test avec le meilleur pipeline : %0.4f ' % accuracy_score(y_test_lem, y_pred) )
        print('Matrice de Confusion :')
        print(confusion_matrix(y_test_lem, y_pred))
        print("Rapport de Classification sur les donénes de Test")
        print(classification_report(y_test_lem,y_pred))   
    else :
        y_pred = best_pipe.predict(X_test_stem)
        roc_score = roc_auc_score(y_test_stem, y_pred)
        print('Score AUC ROC  sur les données de Test avec le meilleur pipeline : %0.4f ' % roc_score )
        print('\n')
        # Afficher d'autres métriques significatives
        print('Score "Accuracy" sur les données de Test avec le meilleur pipeline : %0.4f ' % accuracy_score(y_test_stem, y_pred) )
        print('Matrice de Confusion :')
        print(confusion_matrix(y_test_stem, y_pred))
        print("Rapport de Classification sur les donénes de Test")
        print(classification_report(y_test_stem,y_pred))
    
    #  Suivi du meilleur modèle (AUC ROC la plus élevée)
    if roc_socre > best_roc_auc: 
        best_roc_auc = roc_socre 
        best_gs = gs 
        best_clf = idx 
    

********* Optimisation des Modèles ML et Réglages des Hyperparamètres ....**********

 Nouvelle instane GridSearch...
Pipeline : Stemming - Random Forest/Countvectorizer
------------------------------------------------
Fitting 5 folds for each of 810 candidates, totalling 4050 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 38.6min


# Choix du Meilleur Modèle

In [ ]:
print('\n Le Modèle avec les meilleurs performances est: %s' % grid_dict[best_clf])

# Enregistrer le meilleur pipeline de GridSearch dans un fichier 
dump_file = 'best_gs_pipeline.pkl'
joblib.dump(best_gs, dump_file, compress=1)
print('\nSaved %s grid search pipeline to file: %s' % (grid_dict[best_clf], dump_file))

# CHOIX ET CHARGEMENT DU MODELE

Après avoir choisi notre modèle ainsi que les parameteres les plus optimals, il faut entrainer ce modèle sur toute la base puis sauvegarder le modèle sous forme de fichier .pkl grace au module pickle de Python.
Pickle sert a serialiser, soit convertir une hiérarichie d'obtejts Python en un flux d'octet. Un tel fichier peut donc passer par le "unpickling" pour pouvoir permettre de manipuler le modèle en  python depuis un fihcir de flux d'octet.
Cette étape est indispensable dans la mesure où le but de se projet et d'exposer un modèle ML. On aurait pu entrainer ce modèle dès que l'uilsatuer rentrerait un commentaire à analyser, mais cela serait bcp trop long et innaproprié. Pour cela il est donc indispensable d'enregistrer le modèle afin de deployer facilement un modèle de machine learning sur une interface (pour nous il s'agit d'un site web utilsant la librairie pytho FLASK)
imaginons qu'il s'avere que log regression ait obtneu les meilleur score avec un tfidf sans lem ni stem

https://www.analyticsvidhya.com/blog/2020/04/how-to-deploy-machine-learning-model-flask/

In [7]:
# A mettre à jour cette partie une fois l'entrainement et la comparasion des modèle est terminé
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
import numpy as np


import pickle

def load_model():
    
    X, y = df.StemmedComments, df.NoteLabel
    #a modifier
    vectorizer = TfidfVectorizer(stop_words=used_stop_words,lowercase=True,ngram_range=(1,3),max_features=2000)
    #a modifier avec les patrametress
    
    classifier =  LogisticRegression(max_iter=1000)
   
    sentiment_pipeline = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier',classifier)
    ])
    
    #fiting model with  training data
    sentiment_pipeline.fit(X,y)
    
    #saving model to disk
    pickle.dump(sentiment_pipeline, open('app/model_fr.pkl','wb'))
    
    #loading model to compare the result
    model_fr = pickle.load(open('app/model_fr.pkl','rb'))
   
   

In [8]:
load_model()

## test prediction grace au model chargé

In [31]:
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.snowball import FrenchStemmer
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger


model_fr = pickle.load(open('app/model_fr.pkl','rb'))



#si ni lemm ni stemm
#text = ["bravo"]

#si stemm
text = "honte"
FrenchStemmer = SnowballStemmer("french")
wt = word_tokenize(text)
st = [FrenchStemmer.stem(token) for token in wt ]
st = ' '.join(st)
text = [st]


#si lemm
"""
text = "hontes"
nlp = spacy.load("fr_core_news_sm")
french_lemmatizer = LefffLemmatizer()
nlp.add_pipe(french_lemmatizer, name='lefff')
doc = nlp(text)   
lt=''
for d in doc:
    #print(d.text, d.pos_, d._.lefff_lemma, d.tag_, d.lemma_)
    lt = lt + " " + d.lemma_
text = [lt]
"""
# predict the label using the pipeline
print(str(model_fr.predict(text)[0]))


proba = model_fr.predict_proba(text)[0]
proba_neg = proba[0]
proba_pos = proba[-1]
print(proba_neg)

2020-10-02 07:26:49,744 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-10-02 07:26:49,745 - spacy_lefff.lefff - INFO - Token lefff_lemma already registered
2020-10-02 07:26:49,747 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-10-02 07:26:50,321 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer
1
0.48095825491136135
